In [1]:
import findspark
findspark.init()

In [2]:
import json
# import os

In [3]:
from pyspark import SparkConf, SparkContext, StorageLevel

In [4]:
from projectsupport import *

In [5]:
train_file = "file:///Users/markduan/duan/USC_course/USC_APDS/INF553/project/data/train_review.json"
user_avg_file = "/Users/markduan/duan/USC_course/USC_APDS/INF553/project/data/user_avg.json"
business_avg_file = "/Users/markduan/duan/USC_course/USC_APDS/INF553/project/data/business_avg.json"
user_json = "file:///Users/markduan/duan/USC_course/USC_APDS/INF553/project/data/user.json"
business_json = "file:///Users/markduan/duan/USC_course/USC_APDS/INF553/project/data/business.json"

In [32]:
train_rename_file = "/Users/markduan/duan/USC_course/USC_APDS/INF553/project/model/train_rename.json"

In [6]:
u_table_file = "/Users/markduan/duan/USC_course/USC_APDS/INF553/project/model/u_table.json"
b_table_file = "/Users/markduan/duan/USC_course/USC_APDS/INF553/project/model/b_table.json"

In [7]:
conf = SparkConf() \
    .setAppName("task") \
    .setMaster("local[*]") \
    .set("spark.driver.memory","4g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

In [8]:
raw_data = sc.textFile(train_file).map(json.loads).persist(StorageLevel.MEMORY_AND_DISK)

In [9]:
u_table1 = raw_data.map(lambda x: x['user_id']) \
    .distinct() \
    .collect()

In [10]:
u_set1 = set(u_table1)

In [11]:
b_table1 = raw_data.map(lambda x: x['business_id']) \
    .distinct() \
    .collect()

In [12]:
b_set1 = set(b_table1)

In [13]:
user_avg = getAvg(user_avg_file)
business_avg = getAvg(business_avg_file)

In [14]:
u_set2 = set(user_avg.keys())
b_set2 = set(business_avg.keys())

In [15]:
# def getFriendsAndMe(me, friends):
#     return friends.split(', ') + [me]

In [16]:
# u_table3 = sc.textFile(user_json) \
#     .map(json.loads) \
#     .map(lambda x: (x['user_id'], x['friends'])) \
#     .flatMap(lambda x: getFriendsAndMe(x[0], x[1])) \
#     .distinct() \
#     .collect()

In [17]:
# u_set3 = set(u_table3)

In [18]:
b_table3 = sc.textFile(business_json) \
    .map(json.loads) \
    .map(lambda x: x['business_id']) \
    .collect()

In [19]:
b_set3 = set(b_table3)

In [20]:
u_table = list(u_set1.union(u_set2))
b_table = list(b_set1.union(b_set2).union(b_set3))

In [21]:
u_d = {}
for i in range(len(u_table)):
    k = u_table[i]
    u_d[k] = i
u_d = {u_table[i]: i for i in range(len(u_table))}

In [22]:
b_d = {}
for i in range(len(b_table)):
    k = b_table[i]
    b_d[k] = i

In [23]:
writeDownRenameTable(u_table, u_table_file)
writeDownRenameTable(b_table, b_table_file)

In [26]:
train_0 = sc.textFile(train_file) \
    .map(json.loads) \
    .map(lambda r: (r['user_id'], r['business_id'], r['stars'])) \
    .persist(StorageLevel.MEMORY_AND_DISK)

In [29]:
train_rename = train_0.map(lambda x: (u_d[x[0]], b_d[x[1]], x[2])) \
    .collect()

In [31]:
def saveTrainRename(train_rename, file_path):
    with open(file_path, 'w', encoding='utf-8') as fp:
        json.dump(train_rename, fp)

In [30]:
train_rename

[(2257, 2491, 4.0),
 (34838, 9739, 5.0),
 (88942, 6461, 5.0),
 (73289, 3258, 5.0),
 (77576, 5294, 5.0),
 (85720, 6817, 1.0),
 (6769, 9170, 1.0),
 (178, 12662, 4.0),
 (76177, 10458, 5.0),
 (21800, 5125, 3.0),
 (51128, 7832, 4.0),
 (54959, 10008, 4.0),
 (8697, 5779, 5.0),
 (59842, 11338, 5.0),
 (47011, 11785, 4.0),
 (10212, 6256, 4.0),
 (79285, 8797, 4.0),
 (5442, 5649, 4.0),
 (24965, 12718, 4.0),
 (69397, 8740, 2.0),
 (47931, 4006, 4.0),
 (87113, 13148, 5.0),
 (2026, 6770, 5.0),
 (48368, 388, 5.0),
 (71160, 11217, 5.0),
 (90676, 10300, 5.0),
 (28412, 12661, 5.0),
 (83447, 8995, 4.0),
 (84591, 2323, 4.0),
 (50725, 8074, 1.0),
 (18640, 1884, 4.0),
 (22530, 5267, 5.0),
 (62991, 8675, 5.0),
 (1292, 11662, 4.0),
 (74844, 11110, 1.0),
 (22608, 8405, 5.0),
 (35610, 8068, 3.0),
 (89912, 10671, 2.0),
 (67246, 1830, 5.0),
 (73576, 5586, 5.0),
 (21553, 12619, 4.0),
 (29999, 10933, 2.0),
 (953, 5609, 4.0),
 (63868, 12783, 5.0),
 (84538, 6933, 4.0),
 (4447, 2779, 4.0),
 (37003, 2709, 5.0),
 (28423, 

In [33]:
saveTrainRename(train_rename, train_rename_file)

In [34]:
len(train_rename)

1029758